In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



In [ ]:
# Study data files
crop_production_path = "crop_production_clean.csv"
temp_file_path = "temperature_clean.csv"

In [ ]:
crop_pro_results = pd.read_csv(crop_production_path,encoding="ISO-8859-1")
temp_read_results = pd.read_csv(temp_file_path,encoding="ISO-8859-1")
crop_pro_results =crop_pro_results[ ["Area","Item"] + list(crop_pro_results.loc[:,"Y1961":"Y2013"]) ]
crop_pro_results

#rename the columns for crop file 
renamed_crop_column=[]
for col in crop_pro_results.columns:
    renamed_crop_column.append(f'{col}_crop')
    
crop_pro_results.columns =renamed_crop_column


#compare with temp file country names
area_col=crop_pro_results["Area_crop"].unique()



In [ ]:
# Temperature data set
temp_avg =temp_read_results[["AverageTemperature","year","Country"]]
temp_avg_df=pd.DataFrame(temp_avg).set_index("year")
temp_avg_df = temp_avg_df.reset_index()#.groupby("Country","year").AverageTemperature.mean()
temp_avg_grouped =pd.DataFrame(temp_avg_df).groupby(["Country","year"]).AverageTemperature.mean()
temp_avg_grouped=temp_avg_grouped.unstack()

#reset index to make country a column for merging
temp_avg_grouped=temp_avg_grouped.reset_index()


# Rename the columns for temp file
renamed_column =[]
for column in temp_avg_grouped.columns:
    renamed_column.append(f'{column}Temp')
    
temp_avg_grouped.columns =renamed_column
temp_avg_sorted=temp_avg_grouped.sort_values("CountryTemp")


#crop_results_sorted =crop_pro_results.sort_values("Area_crop")

#Rename country to match our country of focus
crop_country_list =temp_avg_sorted["CountryTemp"].values
for index,value in enumerate(crop_country_list):
    if value =="United States":#"United States of America"
        print(f'index:{index},value:{value}')
        #rename the country to match crop file for merging
        temp_avg_sorted["CountryTemp"][232]="United States of America"


#compare with crop file columns
#country_col =temp_avg_sorted["CountryTemp"].unique()



In [ ]:
#Merge the two data sets
combined_crop_temp_df= crop_pro_results.merge(temp_avg_sorted, how ="outer",left_on='Area_crop', right_on="CountryTemp", suffixes=(False, False))
combined_crop_temp_df =combined_crop_temp_df.dropna()


In [ ]:
# Crop production in the US
usa_prod_df = combined_crop_temp_df.loc[combined_crop_temp_df['Area_crop']=="United States of America"]
usa_prod_df

In [ ]:
Tot_usa_prod_df =usa_prod_df[list(usa_prod_df.loc[:,"Area_crop":"Y2011_crop"])]
Tot_usa_prod_df["Results"]= Tot_usa_prod_df.apply(lambda column: column['Y1961_crop':'Y2011_crop'].sum(),axis = 1)
Tot_usa_prod_df.head()

In [ ]:
sorted_Tot_usa_prod_df = Tot_usa_prod_df.sort_values(by=['Results'], ascending=False)
#maize_new = final_df[['Area', 'maize production']].copy()
clean_df=sorted_Tot_usa_prod_df.head(10)

In [ ]:
clean_df.drop_duplicates()

In [ ]:
clean_df.groupby("Area_crop").sum()